In [1]:
# Don't run this if you already have PyTorch installed.
!pip install torch==2.1.0 torchvision==0.16.0 torchaudio==2.1.0 --index-url https://download.pytorch.org/whl/cu121
# You might need the following before trying to install the packages
!pip install setuptools wheel
# Then proceed to one of the following
!pip install -U audiocraft  # stable release

Looking in indexes: https://download.pytorch.org/whl/cu121
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 GB 692.1 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 87.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 98.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.2/89.2 MB 9.9 MB/s eta 0:00:00
  Attempting uninstall: triton
    Found existing installation: triton 2.2.0
    Uninstalling triton-2.2.0:
      Successfully uninstalled triton-2.2.0
  Attempting uninstall: torch
    Found existing installation: torch 2.2.1+cu121
    Uninstalling torch-2.2.1+cu121:
      Successfully uninstalled torch-2.2.1+cu121
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.17.1+cu121
    Uninstalling torchvision-0.17.1+cu121:
      Successfully uninstalled torchvision-0.17.1+cu121
  Attempting uninstall: torchaudio
    Found existing installation: torchaudio 2.2.1+cu121
    Uninstalling torc

In [2]:
!sudo apt-get install ffmpeg

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 45 not upgraded.


In [1]:
import os
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:

colab_path = "/content/drive/My Drive/Colab Notebooks/AudioLDM"
out_oath = colab_path

In [3]:
import torchaudio
from audiocraft.models import AudioGen
from audiocraft.data.audio import audio_write
import os

model = AudioGen.get_pretrained('facebook/audiogen-medium')
model.set_generation_params(duration=5)  # generate 5 seconds.

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


compression_state_dict.bin:   0%|          | 0.00/236M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/torch/nn/utils/weight_norm.py:30: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")


state_dict.bin:   0%|          | 0.00/3.68G [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.95G [00:00<?, ?B/s]

In [4]:
import torch
def generateAudios(dataset="", prompt_file="", startIndex=0, num_prompts=10):
  directory = colab_path + '/' + dataset
  if not os.path.exists(directory):
      os.makedirs(directory)

  with open(prompt_file) as file:
      prompts = [line.rstrip() for line in file if line != ' ']
  prompts = prompts[startIndex:startIndex + num_prompts]
  print(prompts)
  batch = min(50, num_prompts)
  batches = (len(prompts)+batch-1) // batch

  for batch_i in range(batches):
    start_index = batch_i * batch
    end_index = (batch_i + 1) * batch
    print(start_index, end_index)
    selected = prompts[start_index:end_index]
    wav = model.generate(selected, progress=True)

    for i, one_wav in enumerate(wav):
      # set the seed for generator
      filename = directory + '/'+f"{i+start_index}_0"
      audio_write(filename, one_wav.cpu(), 16000, strategy="loudness", loudness_compressor=True)
      print(filename)
    torch.cuda.empty_cache()

In [8]:
generateAudios(dataset="Urban_Noise", prompt_file="Urban_Noise.txt", startIndex=0, num_prompts=100)

['"Generate the audio of bustling city traffic, with the sounds of cars honking, engines revving, and tires screeching."', '"Create the background noise of a crowded urban street, with people talking, footsteps, and occasional sirens in the distance."', '"Simulate the ambient sounds of a busy subway station, including the rumbling of trains, announcements, and bustling crowds."', '"Generate the audio of a bustling outdoor market, with vendors calling out their products and the chatter of customers."', '"Create the background noise of a construction site, with the sounds of heavy machinery, drilling, and construction workers."', '"Simulate the ambient sounds of a busy city park, including children playing, dogs barking, and people exercising."', '"Generate the audio of a bustling commercial area, with the sounds of people walking, cars passing by, and music from shops."', '"Create the background noise of a busy urban square, with street performers, conversations, and the occasional bicy